In [22]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torch.utils.tensorboard import SummaryWriter
import torchvision


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)
    
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)

In [18]:
learning_rate = 3e-4
z_dim = 64
image_dim = 28*28*1
batch_size = 32
num_epochs = 50

In [19]:
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize((0.5,), (0.5,))
])

In [20]:
dataset = datasets.MNIST(root='MNIST/', transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = torch.optim.Adam(disc.parameters(), lr=learning_rate)
opt_gen = torch.optim.Adam(gen.parameters(), lr=learning_rate)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f'runs/GAN_MNIST/fake')
writer_real = SummaryWriter(f'runs/GAN_MNIST/real')
step = 0

In [21]:
for data, label in loader:
    data = data.view(-1, 784)
print(data.shape)

torch.Size([32, 784])


In [23]:
from tqdm.auto import tqdm
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(tqdm(loader, desc=f'Epoch {epoch+1}', unit='batch')):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        #Train Discriminator: max log(D(real)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2

        disc.zero_grad()

        lossD.backward(retain_graph=True)

        opt_disc.step()

        #Train Generator min log(1 - D(G(z))) == max log(D(G(z))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1
        

Epoch 1:   0%|          | 0/1875 [00:00<?, ?batch/s]

Epoch [0/50] Batch 0/1875                       Loss D: 0.5955, loss G: 0.7351


Epoch 2:   0%|          | 0/1875 [00:00<?, ?batch/s]

Epoch [1/50] Batch 0/1875                       Loss D: 0.3288, loss G: 1.4770


Epoch 3:   0%|          | 0/1875 [00:00<?, ?batch/s]

...

Epoch [47/50] Batch 0/1875                       Loss D: 0.7859, loss G: 0.7953


Epoch 49:   0%|          | 0/1875 [00:00<?, ?batch/s]

Epoch [48/50] Batch 0/1875                       Loss D: 0.6344, loss G: 0.9836


Epoch 50:   0%|          | 0/1875 [00:00<?, ?batch/s]

Epoch [49/50] Batch 0/1875                       Loss D: 0.6646, loss G: 1.0051
